In [ ]:
import pandas as pd



In [1]:
import os

os.chdir("/home/robsyc/Desktop/thesis/MB-VAE-DTI")



In [2]:
import h5torch

from mb_vae_dti.processing import load_h5torch_DTI

In [3]:
file_path = "data/processed/data.h5torch"
with h5torch.File(file_path, "r") as f:
    for key in f.keys():
        print(key)
        print(f[key].keys())

0
<KeysViewHDF5 ['Drug_ID', 'Drug_InChIKey', 'Drug_SMILES', 'Drug_fp']>
1
<KeysViewHDF5 ['Target_AA', 'Target_DNA', 'Target_Gene_name', 'Target_ID', 'Target_RefSeq_ID', 'Target_UniProt_ID', 'Target_fp']>
central
<KeysViewHDF5 ['data', 'indices']>
unstructured
<KeysViewHDF5 ['Y_KIBA', 'Y_pKd', 'Y_pKi', 'in_BindingDB_Kd', 'in_BindingDB_Ki', 'in_DAVIS', 'in_KIBA', 'in_Metz', 'split_cold', 'split_rand']>


In [4]:
with h5torch.File(file_path, "r") as f:
    print(f["1/Target_fp"][153][:20])
    print(f["1/Target_fp"][153].shape)

[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
(4170,)


In [5]:
test_davis_metz = load_h5torch_DTI(
    setting="split_cold",
    split="test",
    datasets=["in_DAVIS", "in_Metz", "in_BindingDB_Kd"]
)
test_davis_metz[153]

Using boolean mask for mapping (7145 indices)
Verified alignment: all unstructured data has 396469 elements


{'central': True,
 '0/Drug_ID': 'D000028',
 '0/Drug_InChIKey': 'XZXHXSATPCNXJR-ZIADKAODSA-N',
 '0/Drug_SMILES': 'COC(=O)c1ccc2c(c1)NC(=O)C2=C(Nc1ccc(N(C)C(=O)CN2CCN(C)CC2)cc1)c1ccccc1',
 '0/Drug_fp': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
 '1/Target_AA': 'MEQPPAPKSKLKKLSEDSLTKQPEEVFDVLEKLGEGSYGSVFKAIHKESGQVVAIKQVPVESDLQEIIKEISIMQQCDSPYVVKYYGSYFKNTDLWIVMEYCGAGSVSDIIRLRNKTLIEDEIATILKSTLKGLEYLHFMRKIHRDIKAGNILLNTEGHAKLADFGVAGQLTDTMAKRNTVIGTPFWMAPEVIQEIGYNCVADIWSLGITSIEMAEGKPPYADIHPMRAIFMIPTNPPPTFRKPELWSDDFTDFVKKCLVKNPEQRATATQLLQHPFIKNAKPVSILRDLITEAMEIKAKRHEEQQRELEEEEENSDEDELDSHTMVKTSVESVGTMRATSTMSEGAQTMIEHNSTMLESDLGTMVINSEDEEEEDGTMKRNATSPQVQRPSFMDYFDKQDFKNKSHENCNQNMHEPFPMSKNVFPDNWKVPQDGDFDFLKNLSLEELQMRLKALDPMMEREIEELRQRYTAKRQPILDAMDAKKRRQQNF',
 '1/Target_DNA': 'AGTAAACTAAAAAAGCTGAGTGAAGACAGTTTGACTAAGCAGCCTGAAGAAGTTTTTGATGTATTAGAGAAGCTTGGAGAAGGGTCTTATGGAAGTGTATTTAAAGCAATACACAAGGAATCCGGTCAAGTTGTCGCAATTAAACAAGTACCTGTTGAATCAGATCTTCAGGAAATAATCAAAGAAATTTCCATAATGCAGCAATGTGACAGCCCATA

In [24]:
test_davis_metz[153]['1/Target_fp']

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [25]:
aa = test_davis_metz[153]['1/Target_AA']
aa

'MEQPPAPKSKLKKLSEDSLTKQPEEVFDVLEKLGEGSYGSVFKAIHKESGQVVAIKQVPVESDLQEIIKEISIMQQCDSPYVVKYYGSYFKNTDLWIVMEYCGAGSVSDIIRLRNKTLIEDEIATILKSTLKGLEYLHFMRKIHRDIKAGNILLNTEGHAKLADFGVAGQLTDTMAKRNTVIGTPFWMAPEVIQEIGYNCVADIWSLGITSIEMAEGKPPYADIHPMRAIFMIPTNPPPTFRKPELWSDDFTDFVKKCLVKNPEQRATATQLLQHPFIKNAKPVSILRDLITEAMEIKAKRHEEQQRELEEEEENSDEDELDSHTMVKTSVESVGTMRATSTMSEGAQTMIEHNSTMLESDLGTMVINSEDEEEEDGTMKRNATSPQVQRPSFMDYFDKQDFKNKSHENCNQNMHEPFPMSKNVFPDNWKVPQDGDFDFLKNLSLEELQMRLKALDPMMEREIEELRQRYTAKRQPILDAMDAKKRRQQNF'

In [7]:
from mb_vae_dti.processing.embed_helper import get_target_fingerprint

aa_fp = get_target_fingerprint(aa)
print(aa_fp.shape)
print(aa_fp[:20])

(4170,)
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [9]:
from mb_vae_dti.processing.embed_helper import get_target_fingerprint

# Example 1: For a function that can process batches:
def batch_target_fingerprint(sequences):
    """Process multiple sequences at once"""
    if isinstance(sequences, str):
        return get_target_fingerprint(sequences)
    else:
        # Process a batch of sequences
        return [get_target_fingerprint(seq) for seq in sequences]

# Using batch processing (process 32 sequences at a time)
add_processed_feature(
    file_path="data/processed/data.h5torch",
    entity_path="1/Target_AA",
    process_func=batch_target_fingerprint,
    feature_name="Target_fp",
    batch_size=32,
    overwrite=True
)

# Example 2: If you had a drug fingerprint function
from mb_vae_dti.processing.embed_helper import get_drug_fingerprint

add_processed_feature(
    file_path="data/processed/data.h5torch",
    entity_path="0/Drug_SMILES",
    process_func=get_drug_fingerprint,
    feature_name="Drug_fp"
)

Processing 2047 entities with batch size 32, output dimension (4170,)


Processing Target_fp batches: 100%|██████████| 64/64 [00:15<00:00,  4.08it/s]


Successfully added Target_fp to data/processed/data.h5torch
Processing 149962 entities with batch size 1, output dimension (2048,)


Processing Drug_fp batches: 100%|██████████| 149962/149962 [02:12<00:00, 1135.22it/s]


Successfully added Drug_fp to data/processed/data.h5torch


---
---

In [6]:
from mb_vae_dti.processing.embed_helper import esm_batch_embed

In [7]:
aa1 = test_davis_metz[153]['1/Target_AA']
aa2 = test_davis_metz[13]['1/Target_AA']

In [8]:
len(aa1), len(aa2)

(491, 765)

In [9]:
result = esm_batch_embed([aa1, aa2])

torch.Size([1, 1, 493, 2560])
torch.Size([1, 1, 2560])
torch.Size([1, 1, 767, 2560])
torch.Size([1, 1, 2560])


In [10]:
result[0]

array([-0.00486905,  0.95842147, -0.0560879 , ..., -0.53510505,
       -0.0018255 ,  0.4314135 ], dtype=float32)

In [11]:
result[1].shape

(2560,)